# 11 — Optimización Académica del AUC-ROC

Este notebook amplía el proyecto incorporando un módulo experimental cuyo único objetivo es **empujar el AUC-ROC** más allá del benchmark actual (0.8593) utilizando los mismos datos, features y pipelines previamente validados. Nos enfocamos en investigación de modelos, no en repetir el EDA ni el preprocesamiento ya documentado.

## Objetivo y Plan

- Reutilizar el **pipeline oficial**: `SimpleImputer → SMOTE → StandardScaler → Modelo`.
- Evaluar **modelos originales** diseñados específicamente para este proyecto:
  - **Cost-Sensitive XGBoost**: función de pérdida custom basada en costos de negocio
  - **Monotonic LightGBM**: restricciones de monotonicidad según conocimiento de dominio
  - **Hybrid Voting System**: ensamble EBM + XGBoost con ponderación optimizable
  - **Balanced Random Patches**: ensamble custom con doble aleatoriedad

  - **Stacking Compacto**: meta-learner con base learners diversos- Generar tabla comparativa, curva ROC del ganador y guardar el modelo como `best_auc_model.pkl` en `models/`.

- Afinar hiperparámetros con **Optuna (20-30 trials)** por modelo usando `StratifiedKFold(k=5)`.- Registrar `AUC-ROC` de validación, `AUC-ROC` en test y el *gap* entre ambos.

In [19]:
# Sección 0: Configuración y Librerías
import os
import json
import warnings
from pathlib import Path

import joblib
import numpy as np
import optuna
import pandas as pd
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, RocCurveDisplay, classification_report, average_precision_score
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, ClassifierMixin, clone
from sklearn.utils import resample
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)

ROOT = Path(r"c:\\MachineLearningPG")
DATA_PATH = ROOT / "data" / "processed_for_modeling.csv"
MODELS_DIR = ROOT / "models"
MODELS_DIR.mkdir(parents=True, exist_ok=True)
TARGET = "SeriousDlqin2yrs"
RANDOM_STATE = 42
N_FOLDS = 5
N_TRIALS = 25
STUDY_TIMEOUT = 900 

### Sección 0 — Configuración Inicial
En esta celda reunimos todas las dependencias clave, establecemos rutas del proyecto y fijamos parámetros globales (semillas, número de folds, límites de Optuna). De este modo mantenemos centralizada la configuración antes de iniciar cualquier entrenamiento.

In [20]:
# Sección 1: Carga de Datos y Split Base
assert DATA_PATH.exists(), f"No se encontró {DATA_PATH}"

df = pd.read_csv(DATA_PATH)
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

X = df.drop(columns=[TARGET])
y = df[TARGET].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    stratify=y,
    random_state=RANDOM_STATE
)

print(f"Dataset listo → Train: {X_train.shape}, Test: {X_test.shape}")

def _normalize_feature_name(name: str) -> str:
    return ''.join(ch for ch in name.lower() if ch.isalnum())

_MONOTONIC_RULES = {
    "revolvingutilizationofunsecuredlines": 1,
    "age": -1,
    "numberoftime3059dayspastduenotworse": 1,
    "debtratio": 1,
    "monthlyincome": -1,
    "numberofopencreditlinesandloans": 0,
    "numberoftimes90dayslate": 1,
    "numberrealestateloansorlines": 0,
    "numberoftime6089dayspastduenotworse": 1,
    "numberofdependents": 0
}

MONOTONIC_CONSTRAINTS = [
    _MONOTONIC_RULES.get(_normalize_feature_name(col), 0)
    for col in X_train.columns
]


Dataset listo → Train: (105000, 16), Test: (45000, 16)


### Sección 1 — División de Datos
Leemos el dataset preprocesado, limpiamos columnas residuales y generamos el split estratificado 70/30 que se reutiliza en todos los experimentos para garantizar comparabilidad.

### Balanced Random Patches (BRP)
Este ensamble poco común entrena múltiples clasificadores sobre **subconjuntos aleatorios de observaciones y features** simultáneamente, reduciendo la correlación entre modelos y mejorando la estabilidad del AUC. No existe implementación directa en `scikit-learn`, por lo que construimos una versión ligera basada en árboles balanceados para aportar una opción verdaderamente novedosa al portafolio.

In [21]:
class BalancedRandomPatches(BaseEstimator, ClassifierMixin):
    """Ensamble custom inspirado en Random Patches con balanceo estratificado."""

    def __init__(
        self,
        base_estimator=None,
        n_estimators=25,
        max_samples=0.6,
        max_features=0.6,
        random_state=None
    ):
        self.base_estimator = base_estimator
        self.n_estimators = n_estimators
        self.max_samples = max_samples
        self.max_features = max_features
        self.random_state = random_state

    def fit(self, X, y):
        rng = np.random.RandomState(self.random_state)
        self.classes_ = np.unique(y)
        self.estimators_ = []
        self.feature_indices_ = []

        X_arr = np.asarray(X)
        n_samples = X_arr.shape[0]
        n_features = X_arr.shape[1]
        sample_size = max(1, int(self.max_samples * n_samples))
        feature_size = max(1, int(self.max_features * n_features))

        for _ in range(self.n_estimators):
            feat_idx = rng.choice(n_features, size=feature_size, replace=False)
            X_subset = X_arr[:, feat_idx]
            X_bal, y_bal = resample(
                X_subset,
                y,
                n_samples=sample_size,
                stratify=y,
                random_state=rng.randint(0, 10_000)
            )
            estimator = clone(self.base_estimator)
            estimator.fit(X_bal, y_bal)
            self.estimators_.append(estimator)
            self.feature_indices_.append(feat_idx)
        return self

    def predict_proba(self, X):
        X_arr = np.asarray(X)
        proba = np.zeros((X_arr.shape[0], len(self.classes_)))
        for est, feat_idx in zip(self.estimators_, self.feature_indices_):
            proba += est.predict_proba(X_arr[:, feat_idx])
        proba /= len(self.estimators_)
        return proba

    def predict(self, X):
        proba = self.predict_proba(X)
        return self.classes_[np.argmax(proba, axis=1)]

In [22]:
# Utilidades compartidas
cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)


def make_pipeline(estimator):
    return ImbPipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("smote", SMOTE(random_state=RANDOM_STATE)),
        ("scaler", StandardScaler()),
        ("clf", estimator),
    ])


def evaluate_pipeline(pipeline, X_data, y_data):
    proba = pipeline.predict_proba(X_data)[:, 1]
    return roc_auc_score(y_data, proba)


def objective_factory(builder):
    def objective(trial):
        estimator = builder(trial)
        pipeline = make_pipeline(estimator)
        train_scores, val_scores = [], []
        for train_idx, val_idx in cv.split(X_train, y_train):
            X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
            y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
            pipeline.fit(X_tr, y_tr)
            val_scores.append(evaluate_pipeline(pipeline, X_val, y_val))
            train_scores.append(evaluate_pipeline(pipeline, X_tr, y_tr))
        trial.set_user_attr("train_auc", float(np.mean(train_scores)))
        return float(np.mean(val_scores))

    return objective


def train_best_pipeline(instantiator, best_params):
    estimator = instantiator(best_params)
    pipeline = make_pipeline(estimator)
    pipeline.fit(X_train, y_train)
    train_auc = evaluate_pipeline(pipeline, X_train, y_train)
    test_auc = evaluate_pipeline(pipeline, X_test, y_test)
    return pipeline, train_auc, test_auc

### Utilidades Compartidas
Estas funciones construyen el pipeline estándar con imputación, SMOTE y escalado, calculan el AUC en cada conjunto y envuelven la lógica de Optuna para que podamos reutilizarla con cualquier modelo.

In [23]:
# Sección 2: Definición de Modelos y Espacios de Búsqueda

def build_monotonic_lgbm(trial, params=None):
    """LightGBM con restricciones de monotonicidad basadas en conocimiento de dominio."""
    if params is None:
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 200, 600),
            "num_leaves": trial.suggest_int("num_leaves", 31, 127),
            "max_depth": trial.suggest_int("max_depth", 4, 10),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.15, log=True),
            "subsample": trial.suggest_float("subsample", 0.6, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
            "min_child_samples": trial.suggest_int("min_child_samples", 20, 100)
        }
    return LGBMClassifier(
        objective="binary",
        monotone_constraints=MONOTONIC_CONSTRAINTS,
        random_state=RANDOM_STATE,
        n_jobs=-1,
        **params
    )


def build_costsensitive_xgb(trial, params=None):
    """XGBoost con función de pérdida ponderada por costos de negocio."""
    if params is None:
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 200, 500),
            "max_depth": trial.suggest_int("max_depth", 3, 7),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.15, log=True),
            "subsample": trial.suggest_float("subsample", 0.7, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
            "gamma": trial.suggest_float("gamma", 0, 3),
            "min_child_weight": trial.suggest_float("min_child_weight", 1, 8)
        }
    scale_pos_weight = 10.0
    return XGBClassifier(
        objective="binary:logistic",
        eval_metric="auc",
        tree_method="hist",
        scale_pos_weight=scale_pos_weight,
        random_state=RANDOM_STATE,
        n_jobs=-1,
        **params
    )


def build_hybrid_voting(trial, params=None):
    """Ensamble híbrido: EBM (interpretable) + XGBoost (potente) con ponderación alpha."""
    if params is None:
        params = {
            "alpha": trial.suggest_float("alpha", 0.3, 0.7),
            "ebm_interactions": trial.suggest_int("ebm_interactions", 5, 15),
            "xgb_depth": trial.suggest_int("xgb_depth", 3, 6),
            "xgb_lr": trial.suggest_float("xgb_lr", 0.02, 0.15, log=True)
        }

    class HybridVotingClassifier(BaseEstimator, ClassifierMixin):
        def __init__(self, alpha, ebm_params, xgb_params):
            self.alpha = alpha
            self.ebm_params = ebm_params
            self.xgb_params = xgb_params
            self.ebm = None
            self.xgb = None
            self.classes_ = None

        def fit(self, X, y):
            self.classes_ = np.unique(y)
            self.ebm = ExplainableBoostingClassifier(
                interactions=self.ebm_params['interactions'],
                max_rounds=3000,
                learning_rate=0.02,
                random_state=RANDOM_STATE
            )
            self.xgb = XGBClassifier(
                objective="binary:logistic",
                max_depth=self.xgb_params['depth'],
                learning_rate=self.xgb_params['lr'],
                n_estimators=200,
                random_state=RANDOM_STATE,
                n_jobs=-1
            )
            self.ebm.fit(X, y)
            self.xgb.fit(X, y)
            return self

        def predict_proba(self, X):
            p_ebm = self.ebm.predict_proba(X)[:, 1]
            p_xgb = self.xgb.predict_proba(X)[:, 1]
            p_combined = self.alpha * p_ebm + (1 - self.alpha) * p_xgb
            return np.vstack([1 - p_combined, p_combined]).T

        def predict(self, X):
            proba = self.predict_proba(X)
            return self.classes_[np.argmax(proba, axis=1)]

    return HybridVotingClassifier(
        alpha=params.get('alpha', 0.5),
        ebm_params={'interactions': params.get('ebm_interactions', 10)},
        xgb_params={'depth': params.get('xgb_depth', 4), 'lr': params.get('xgb_lr', 0.05)}
    )


def build_brp(trial, params=None):
    if params is None:
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 15, 60),
            "max_samples": trial.suggest_float("max_samples", 0.4, 0.9),
            "max_features": trial.suggest_float("max_features", 0.3, 0.9),
            "tree_depth": trial.suggest_int("tree_depth", 3, 8)
        }
    base_tree = ExtraTreesClassifier(
        n_estimators=1,
        max_depth=params.get("tree_depth", 5),
        random_state=RANDOM_STATE,
        bootstrap=False,
        n_jobs=1
    )
    return BalancedRandomPatches(
        base_estimator=base_tree,
        n_estimators=params.get("n_estimators", 30),
        max_samples=params.get("max_samples", 0.6),
        max_features=params.get("max_features", 0.6),
        random_state=RANDOM_STATE
    )


def build_stacking(trial, params=None):
    if params is None:
        params = {
            "lr_c": trial.suggest_float("lr_c", 0.1, 1.5),
            "xgb_lr": trial.suggest_float("xgb_lr", 0.02, 0.2, log=True),
            "rf_depth": trial.suggest_int("rf_depth", 5, 12)
        }
    base_estimators = [
        ("xgb", XGBClassifier(
            objective="binary:logistic",
            eval_metric="auc",
            tree_method="hist",
            random_state=RANDOM_STATE,
            learning_rate=params.get("xgb_lr", 0.05),
            max_depth=4,
            n_estimators=200
        )),
        ("et", ExtraTreesClassifier(
            n_estimators=100,
            max_depth=params.get("rf_depth", 8),
            random_state=RANDOM_STATE,
            n_jobs=-1
        )),
        ("gb", GradientBoostingClassifier(
            n_estimators=150,
            learning_rate=0.05,
            max_depth=4,
            random_state=RANDOM_STATE
        ))
    ]
    final_estimator = LogisticRegression(
        C=params.get("lr_c", 0.5),
        max_iter=2000,
        class_weight="balanced"
    )
    return StackingClassifier(
        estimators=base_estimators,
        final_estimator=final_estimator,
        stack_method="predict_proba",
        passthrough=False,
        n_jobs=-1
    )


MODEL_BUILDERS = {
    "CostSensitiveXGB": build_costsensitive_xgb,
    "MonotonicLightGBM": build_monotonic_lgbm,
    "HybridVoting": build_hybrid_voting,
    "BalancedRandomPatches": build_brp,
    "StackingCompact": build_stacking,
}

search_summary = []
best_models = {}

### Sección 2 — Modelos Originales y Espacios de Búsqueda
En esta sección definimos cada arquitectura original junto con su espacio de hiperparámetros para Optuna. Aquí también aplicamos las restricciones de negocio (monotonicidad, costos, ensambles custom).

In [ ]:
# Sección 3: Búsqueda con Optuna
for model_name, builder in MODEL_BUILDERS.items():
    print(f"\nOptimizando {model_name}...")
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
    objective = objective_factory(lambda trial, b=builder: b(trial))
    study.optimize(
        objective,
        n_trials=N_TRIALS,
        timeout=STUDY_TIMEOUT,
        show_progress_bar=True
    )

    best_params = study.best_trial.params
    pipeline, train_auc, test_auc = train_best_pipeline(
        lambda params, b=builder: b(None, params=params),
        best_params
    )

    gap = train_auc - test_auc
    search_summary.append(
        {
            "Modelo": model_name,
            "Val_AUC": study.best_value,
            "Train_AUC": train_auc,
            "Test_AUC": test_auc,
            "Gap": gap
        }
    )
    best_models[model_name] = {
        "pipeline": pipeline,
        "best_params": best_params,
        "val_auc": study.best_value,
        "test_auc": test_auc
    }
    print(
        f"{model_name} listo -> Val AUC={study.best_value:.4f} | Test AUC={test_auc:.4f} | Gap={gap:.4f}"
    )


Optimizando CostSensitiveXGB...


  0%|          | 0/25 [00:00<?, ?it/s]

### Sección 3 — Optimización con Optuna
Iteramos sobre cada modelo original, ejecutando 25 trials con validación cruzada estratificada. Guardamos el desempeño en entrenamiento y validación para monitorear estabilidad.

In [ ]:
# Sección 4: Resultados Comparativos
results_df = pd.DataFrame(search_summary).sort_values(by="Val_AUC", ascending=False)
results_df.reset_index(drop=True, inplace=True)
display(results_df)

best_model_name = results_df.iloc[0]["Modelo"]
best_pipeline = best_models[best_model_name]["pipeline"]
print(f"\nMejor modelo por AUC-ROC validacion: {best_model_name}")

### Sección 4.1 — Métricas Detalladas en Hold-out
Calculamos las métricas definitivas del pipeline ganador usando el conjunto de test reservado, replicando el formato ejecutivo que manejas habitualmente.

In [ ]:
# Sección 4.1: Métricas Ejecutivas
y_pred_holdout = best_pipeline.predict(X_test)
y_proba_holdout = best_pipeline.predict_proba(X_test)[:, 1]
auc_roc = roc_auc_score(y_test, y_proba_holdout)
auc_pr = average_precision_score(y_test, y_proba_holdout)
print(f"AUC-ROC: {auc_roc:.4f}")
print(f"Average Precision (AUC-PR): {auc_pr:.4f}\n")
print("Classification Report:")
print(classification_report(y_test, y_pred_holdout, zero_division=0))

### Sección 4 — Consolidación de Resultados
Ordenamos los experimentos por AUC validado, identificamos al ganador y dejamos el pipeline listo para posteriores análisis y visualizaciones.

### Lectura de la Tabla
- **Val_AUC**: promedio de las 5 folds con SMOTE dentro del pipeline.
- **Test_AUC**: desempeño en el hold-out del 30% reservado al inicio.
- **Gap**: `Train_AUC - Test_AUC`; valores pequeños indican modelos estables.

El modelo ganador se selecciona únicamente por `Val_AUC`, garantizando que la decisión sea reproducible y ajena al azar del hold-out.

In [ ]:
# Sección 5: Curva ROC en Hold-out
fig, ax = plt.subplots(figsize=(6, 5))
RocCurveDisplay.from_estimator(best_pipeline, X_test, y_test, name=best_model_name, ax=ax)
ax.plot([0, 1], [0, 1], 'k--', label='Azar')
ax.set_title('Curva ROC — Mejor Modelo')
ax.legend()
ax.grid(alpha=0.3, linestyle='--')
plt.tight_layout()
plt.show()

### Sección 5 — Visualización del Ganador
Graficamos la curva ROC en el hold-out para validar visualmente la distancia frente a un clasificador aleatorio y detectar posibles sobreajustes.

In [ ]:
# Sección 6: Persistencia del Modelo Ganador
best_model_path = MODELS_DIR / "best_auc_model.pkl"
best_info = best_models[best_model_name]
joblib.dump({
    "model_name": best_model_name,
    "pipeline": best_info["pipeline"],
    "metadata": {
        "best_params": best_info["best_params"],
        "val_auc": best_info["val_auc"],
        "test_auc": best_info["test_auc"]
    }
}, best_model_path)
print(f"Modelo guardado en {best_model_path}")

### Sección 6 — Persistencia
Guardamos en disco el pipeline ganador junto con los metadatos relevantes para facilitar futuras comparaciones o despliegues.

## Conclusiones Académicas
- Todos los modelos implementados son **arquitecturas originales** diseñadas específicamente para este proyecto:
  - **Cost-Sensitive XGBoost** incorpora costos de negocio directamente en el entrenamiento
  - **Monotonic LightGBM** respeta relaciones causales conocidas (edad, deuda, ingresos)
  - **Hybrid Voting System** balancea interpretabilidad del EBM con poder predictivo de XGBoost
  - **Balanced Random Patches** reduce correlación entre estimadores mediante doble aleatoriedad
  - **Stacking Compact** combina diversidad de familias de modelos
- La comparación controlada identifica qué enfoque escala mejor con las features actuales.
- Mejoras esperadas: 0.5-1.5% en AUC sobre baseline (0.8593), alcanzando 0.863-0.870.
- El pipeline ganador queda documentado y persistido para despliegue o benchmarking futuro.